In [18]:
import sys
import os

sys.path.insert(0, 'compiled_protobufs')

import pandas as pd
qrels = "/home/ubuntu/task-search-quality/measurements/judgments/diy.qrels"

diy_queries = pd.read_csv('queries/diy.csv')

from google.protobuf.json_format import MessageToDict
import json
from google.protobuf.json_format import Parse
from pyserini.search import LuceneSearcher
from taskmap_pb2 import TaskMap

In [25]:
def get_taskgraph(id, domain):
    taskmap_index_path = os.path.join(os.getcwd(), "indexes", domain, "system_index_sparse")
    searcher = LuceneSearcher(index_dir=taskmap_index_path)
    id = id.replace('\n','')
    doc = searcher.doc(docid=id)
    taskmap_json = json.loads(doc.raw())['recipe_document_json']
    taskmap = Parse(json.dumps(taskmap_json), TaskMap())
    return taskmap

In [27]:
with open(qrels, "r") as f:
    lines = []
    for line in f:
        lines.append(line.strip().split(" "))
    df = pd.DataFrame(lines, columns=["qid", "run", "docid", "relevance"])
    
def get_raw(x):
    qid = x["qid"]
    query_info = diy_queries[diy_queries["id"] == qid].iloc[0]
    return query_info["raw query"]

def get_title(x):
    taskmap = get_taskgraph(x["docid"], "diy")
    return taskmap.title

def get_url(x):
    taskmap = get_taskgraph(x["docid"], "diy")
    return taskmap.source_url

df["query_raw"] = df.apply(lambda x: get_raw(x), axis=1)
df["doc_title"] = df.apply(lambda x: get_title(x), axis=1)
df["doc_url"] = df.apply(lambda x: get_url(x), axis=1)
df.head()
df.to_csv("/home/ubuntu/task-search-quality/measurements/annotations/diy.csv", index=False)

In [12]:
path = "/home/ubuntu/task-search-quality/measurements/diy/filled_annotations/missing-annotations.csv"
qrels_path = "/home/ubuntu/task-search-quality/measurements/judgments/diy2.qrels"

with open(path, "r") as f:
    annotations = [a.strip().split(',') for a in f]

with open(qrels_path, "w") as f:
    for a in annotations[1:]:
        q_id, doc_id = a[0], a[1]
        rel = a[-1]
        f.write(f'{q_id} Q0 {doc_id} {rel}\n')

['query-0', 'cooking+recipe1m+06c38e07b3533c34738b86f379aafe5c', 'how to cook a spatchcocked turkey', 'Turducken Moist and Flavorful Good Eats!', '0']
